# Experiments in Modeling Issue Segmentation

## Load Libraries and Datasets

In [2]:
import os
import re
import pandas as pd
from tqdm import tqdm
import altair as alt
alt.data_transformers.disable_max_rows()
from collections import deque
import warnings
warnings.filterwarnings('ignore')
from rich.console import Console
from rich.table import Table
import numpy as np
import scipy.stats as stats
from minineedle import needle, core
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
console = Console()
import sys

sys.path.append("..")
from segmentation_scripts.utils import read_csv_file, get_data_directory_path

In [3]:
def generate_table(df, table_title):
    # Create a Rich Table
    table = Table(title=table_title)
    columns = df.columns
    for column in columns:
        table.add_column(column.replace("_", " ").capitalize(), justify="center", style="cyan", no_wrap=True)    

    # Add rows to the table
    for _, row in df.iterrows():
        table.add_row(*[str(value) if pd.notna(value) else "" for value in row])

    # Print the table
    console.print(table)

In [9]:
def filter_integers(token: str) -> bool:
	"""Check if the token is an integer."""
	return bool(re.match(r'^\d+$', token))

def calculate_digit_coverage(rows):
    """Calculate the number of digits in the given rows."""
    number_of_digits = rows['implied_zero'].notna().sum()
    return number_of_digits

def clean_digits(df: pd.DataFrame) -> pd.DataFrame:
	"""Clean and filter digit tokens in the DataFrame while retaining non-digit pages."""
	max_possible_number = df.page_number.max()
	
	df['token'] = df['token'].astype(str)
	if 'volume_name' in df.columns:
		df['volume_number'] = df['volume_number'].fillna(0)
	
	# Identify pages with digit tokens
	subset_digits = df[df['token'].str.isdigit()].copy()
	possible_pages = subset_digits[subset_digits['token'].apply(filter_integers)].copy()
	# Use the smaller subset if possible_pages is smaller
	console.print(f"Number of possible pages {len(possible_pages)} vs subset_digits {len(subset_digits)}")
	if len(possible_pages) < len(subset_digits):
		subset_digits = possible_pages

	# Identify pages without digit tokens
	non_digits_pages = df[(~df['token'].str.isdigit()) & (~df.page_number.isin(subset_digits.page_number))].copy()
	
	# just take the first page of non_digits_pages
	non_digits_pages = non_digits_pages.groupby('page_number').first().reset_index()
	console.print(f"Number of digits in this volume: {len(subset_digits)}")
	console.print(f"Number of non-digit pages in this volume: {len(non_digits_pages)}")
	
	subset_digits['number'] = subset_digits['token'].astype(int, errors='ignore')
	# max_possible_number = max_page + 25
	filtered_subset_digits = subset_digits[(subset_digits['number'] < max_possible_number) & (subset_digits['number'] <= subset_digits.page_number)].copy()
	non_filtered_subset_digits = subset_digits[(subset_digits['number'] >= max_possible_number) & (~subset_digits.page_number.isin(filtered_subset_digits.page_number))].groupby('page_number').first().reset_index()
	console.print(f"Number of digits in this volume after filtering for max page length: {len(filtered_subset_digits)}")
	console.print(f"Number of pages without digits in this volume after filtering for max page length: {len(non_filtered_subset_digits)}")
	
	# Calculate implied zero only for digit pages
	filtered_subset_digits['implied_zero'] = filtered_subset_digits['page_number'].astype(int) - filtered_subset_digits['number']
	
	final_subset_digits = filtered_subset_digits[filtered_subset_digits['implied_zero'] >= 0]
	console.print(f"Number of digits in this volume after filtering for max page length and implied zero: {len(final_subset_digits)}")
	remaining_missing_pages = df[(~df.page_number.isin(final_subset_digits.page_number)) & (~df.page_number.isin(non_digits_pages.page_number))].copy()
	remaining_missing_pages = remaining_missing_pages.groupby('page_number').first().reset_index()
	console.print(f"Number of pages without digits in this volume after filtering for max page length and digit pages: {len(remaining_missing_pages)}")
	
	# Merge non-digit pages back into the DataFrame
	non_digits_pages['page_type'] = 'non_digit'
	remaining_missing_pages['page_type'] = 'negative_na_digit'
	final_subset_digits['page_type'] = 'digit'
	non_filtered_subset_digits['page_type'] = 'digit_too_large'
	full_df_with_digits = pd.concat([final_subset_digits, non_digits_pages, remaining_missing_pages, non_filtered_subset_digits]).sort_values(by=['page_number']).reset_index(drop=True)
	
	console.print(f"Number of pages after including non-digit pages: {full_df_with_digits.page_number.nunique()}")

	if full_df_with_digits.page_number.nunique() != df.page_number.nunique():
		added_pages = df[~df.page_number.isin(full_df_with_digits.page_number)].copy()
		added_pages = added_pages.groupby('page_number').first().reset_index()
		added_pages['page_type'] = 'added'
		full_df_with_digits = pd.concat([full_df_with_digits, added_pages]).sort_values(by=['page_number']).reset_index(drop=True)
		console.print(f"Number of pages after including added pages: {full_df_with_digits.page_number.nunique()}")

	# Calculate the number of digits per page
	tqdm.pandas(desc="Calculating digits per page")
	digits_per_page = full_df_with_digits.groupby('page_number').progress_apply(calculate_digit_coverage).reset_index(name='digits_per_page')
	full_df_with_digits = full_df_with_digits.merge(digits_per_page, on='page_number', how='left')
	
	return full_df_with_digits

In [63]:
def process_file(file_path):
	full_df = read_csv_file(file_path)
	console.print(f"Volume has this many tokens: {len(full_df)}")
	console.print(f"Volume has this many issues: {full_df.start_issue.nunique()}")
	console.print(f"Volume has this many pages: {full_df.page_number.nunique()}")
	# Factorize the 'issue_number' column to create 'actual_issue_number'
	full_df = full_df.sort_values(by=['page_number'])
	full_df = full_df.rename(columns={'issue_number': 'original_issue_number', 'page_number': 'original_page_number'})
	full_df['temp_issue_number'] = pd.factorize(full_df['original_issue_number'])[0]
	# Factorize the 'original_page_number' column
	factorized_values, unique_values = pd.factorize(full_df['original_page_number'])

	# Adjust the factorized values to start at 1
	full_df['page_number'] = factorized_values + 1
	# Expand count column
	expanded_df = full_df.loc[full_df.index.repeat(full_df['count'])].reset_index(drop=True)
	console.print(f"Expanded volume has this many tokens: {len(expanded_df)}")

	# Calculate the number of tokens per page
	tokens_per_page = expanded_df.groupby('page_number').size().reset_index(name='tokens_per_page')

	# Merge the token counts back into the original DataFrame
	expanded_df = expanded_df.merge(tokens_per_page, on='page_number', how='left')
	missing_pages = full_df[~full_df.page_number.isin(expanded_df.page_number.unique())]
	expanded_df = pd.concat([expanded_df, missing_pages], ignore_index=True)
	expanded_df = expanded_df.reset_index(drop=True)

	annotated_df = full_df[['page_number', 'start_issue', 'end_issue', 'type_of_page']].drop_duplicates()

	# Group by 'start_issue' and aggregate
	grouped_df = annotated_df.groupby('start_issue').agg(
		first_page=('page_number', 'min'),
		last_page=('page_number', 'max'),
		number_of_pages=('page_number', 'count')
	).reset_index()
	grouped_df = grouped_df.sort_values(by='first_page')

	generate_table(grouped_df, "Grouped by 'start_issue'")

	# Example usage
	subset_digits = clean_digits(expanded_df)
	subset_digits = subset_digits.sort_values(by=['page_number'])
	counts_per_annotated_issue = subset_digits.start_issue.value_counts().reset_index()

	generate_table(counts_per_annotated_issue, "Counts per Annotated Issue")
	return expanded_df, subset_digits, grouped_df

def calculate_coverage(expanded_df, subset_digits, grouped_df):

	distribution_df = subset_digits[['page_number', 'tokens_per_page', 'digits_per_page', 'start_issue']].drop_duplicates()
	distribution_df = distribution_df.sort_values(by='page_number').reset_index(drop=True)
	distribution_df['digit_ratio'] = distribution_df['digits_per_page'] / distribution_df['tokens_per_page']

	# Calculate the mean digit ratio per issue
	mean_digit_ratio_per_issue = distribution_df.groupby('start_issue')['digit_ratio'].mean().reset_index(name='mean_digit_ratio')
	generate_table(mean_digit_ratio_per_issue, "Mean Digit Ratio per Issue")

	# Calculate tokens_per_page
	tokens_per_page_df = expanded_df.groupby(['start_issue', 'page_number']).size().reset_index(name='tokens_per_page')
	# Calculate the 10th percentile
	tenth_percentile = tokens_per_page_df['tokens_per_page'].quantile(0.5)
	periodical_title = expanded_df['periodical_name'].unique()[0]
	volume_number = expanded_df['volume_directory'].unique()[0]
	# Create the base chart
	base = alt.Chart(tokens_per_page_df).mark_line(point=True).encode(
		x=alt.X("page_number:Q", scale=alt.Scale(zero=False)),
		y=alt.Y('tokens_per_page:Q', scale=alt.Scale(zero=False)),
		color='start_issue:N',
		tooltip=['page_number', 'tokens_per_page', 'start_issue'],
	).properties(
		width=400,
		height=200,
		title=f'Tokens per Page per Issue for {periodical_title} Volume {volume_number}'	
	)

	# Add the quantile line
	quantile_line = alt.Chart(pd.DataFrame({'y': [tenth_percentile]})).mark_rule(color='red').encode(
		y=alt.Y('y:Q', axis=alt.Axis(title=None))
	)

	# Combine the base chart and the quantile line
	chart = base + quantile_line

	# Calculate the 10th percentile
	tenth_percentile = tokens_per_page_df['tokens_per_page'].quantile(0.5)

	# Filter the DataFrame to get the rows in the lowest ten percent
	lowest_ten_percent_tokens_per_page_df = tokens_per_page_df[tokens_per_page_df['tokens_per_page'] <= tenth_percentile]

	def check_if_actual_issue(row, grouped_df):
		subset_grouped_df = grouped_df[grouped_df.first_page == row.page_number]
		row['actual_issue'] = True if len(subset_grouped_df) > 0 else False
		return row

	# Check if the page is an actual issue
	tqdm.pandas(desc="Checking if actual issue")
	lowest_ten_percent_tokens_per_page_df = lowest_ten_percent_tokens_per_page_df.progress_apply(check_if_actual_issue, args=(grouped_df,), axis=1)

	lowest_ten_percent_tokens_per_page_df = lowest_ten_percent_tokens_per_page_df.sort_values(by=['page_number','tokens_per_page'], ascending=[True, False])

	console.print(f"We are missing the following issues with this approach {grouped_df[(~grouped_df.start_issue.isin(lowest_ten_percent_tokens_per_page_df[lowest_ten_percent_tokens_per_page_df.actual_issue == True].start_issue))].start_issue.unique()}", style="bright_red")
	generate_table(lowest_ten_percent_tokens_per_page_df[lowest_ten_percent_tokens_per_page_df.actual_issue == True], "Lowest Ten Percent of Tokens per Page")
	return distribution_df, mean_digit_ratio_per_issue, chart, grouped_df[(~grouped_df.start_issue.isin(lowest_ten_percent_tokens_per_page_df[lowest_ten_percent_tokens_per_page_df.actual_issue == True].start_issue))].start_issue.unique()

In [55]:
data_directory_path = get_data_directory_path()
preidentified_periodicals_df = read_csv_file(os.path.join(data_directory_path, "HathiTrust-pcc-datasets", "datasets", "preidentified_periodicals_with_full_metadata.csv"))

In [56]:
preidentified_periodicals_df[0:1][['publication_directory', 'volume_directory']].to_dict(orient='records')

[{'publication_directory': 'datasets/ht_ef_datasets/la_documentation_arabe',
  'volume_directory': 'inu_30000093395964'}]

In [6]:
# Count the number of matching files
matching_files = []
for directory, _, files in tqdm(os.walk("../datasets/annotated_ht_ef_datasets/"), desc="Counting matching files"):
	for file in files:
		if file.endswith(".csv") and 'individual' in file:
			if os.path.exists(os.path.join(directory, file)):
				publication_name = directory.split("/")[-2]
				matching_files.append({"file": file, "directory": directory, "file_path": os.path.join(directory, file), "periodical_title": publication_name})
matching_files_df = pd.DataFrame(matching_files)
console.print(f"Found {len(matching_files_df)} matching files.", style="bright_green")

Counting matching files: 0it [00:00, ?it/s]

Counting matching files: 95it [00:00, 1240.37it/s]


Found 85 matching files.

In [8]:
matching_files_df.periodical_title.value_counts()

periodical_title
lotus                           27
arab_observer_and_the_scribe    23
tricontinental                  12
afro_asian_bulletin              9
liberator                        6
afro_asian_and_world_affairs     3
afro_asian_peoples               2
solidarity_aapso                 2
scribe                           1
Name: count, dtype: int64

In [67]:
matching_files_df[matching_files_df.periodical_title == "solidarity_aapso"]

,file,directory,file_path,periodical_title
74,uc1_c2693407_annotated_individual_tokens.csv,../datasets/annotated_ht_ef_datasets/solidarit...,../datasets/annotated_ht_ef_datasets/solidarit...,solidarity_aapso
75,inu_30000003214412_annotated_individual_tokens...,../datasets/annotated_ht_ef_datasets/solidarit...,../datasets/annotated_ht_ef_datasets/solidarit...,solidarity_aapso


In [65]:
missing_issues_df = pd.DataFrame(missing_issues)

missing_issues_df = missing_issues_df[missing_issues_df.missing_issues.str.len() > 0]
missing_issues_df.to_csv("missing_issues.csv", index=False)

In [66]:
missing_issues_df

,file_path,missing_issues,periodical_title
3,../datasets/annotated_ht_ef_datasets/liberator...,"[1968-05-01, 1968-06-01]",liberator
8,../datasets/annotated_ht_ef_datasets/arab_obse...,[1966-09-26],arab_observer_and_the_scribe
18,../datasets/annotated_ht_ef_datasets/arab_obse...,"[1963-07-15, 1963-09-30]",arab_observer_and_the_scribe
20,../datasets/annotated_ht_ef_datasets/arab_obse...,"[1965-08-02, 1965-08-16, 1965-08-30, 1965-09-06]",arab_observer_and_the_scribe
27,../datasets/annotated_ht_ef_datasets/arab_obse...,[1963-11-04],arab_observer_and_the_scribe
58,../datasets/annotated_ht_ef_datasets/tricontin...,[1966-08-01],tricontinental
61,../datasets/annotated_ht_ef_datasets/tricontin...,[1980-05-01],tricontinental
69,../datasets/annotated_ht_ef_datasets/afro_asia...,"[1964-09-01, 1965-12-01, 1966-09-01, 1966-12-01]",afro_asian_and_world_affairs
84,../datasets/annotated_ht_ef_datasets/afro_asia...,[1962-09-01],afro_asian_bulletin


In [57]:
# alt.vconcat(*charts)

In [64]:
charts = []
missing_issues = []
for index, row in matching_files_df.iterrows():
	file = row['file']
	directory = row['directory']
	file_path = row['file_path']
	console.print(f"Processing file: {file_path}. Number {index} out of {len(matching_files_df)}", style="bright_white")
	expanded_df, subset_digits, grouped_df = process_file(file_path)
	distribution_df, mean_digit_ratio_per_issue, chart, missing_issues_results = calculate_coverage(expanded_df, subset_digits, grouped_df)
	charts.append(chart)
	missing_issues.append({"file_path": file_path, "missing_issues": missing_issues_results, "periodical_title": row['periodical_title']})
	

Processing file: 
../datasets/annotated_ht_ef_datasets/liberator/mdp_39015038717792/mdp_39015038717792_annotated_individual_tokens.cs
v. Number 0 out of 85

Volume has this many tokens: 21002

Volume has this many issues: 3

Volume has this many pages: 72

Expanded volume has this many tokens: 41781

...

 Counts per Annotated  
         Issue         
┏━━━━━━━━━━━━━┳━━━━━━━┓
┃ Start issue ┃ Count ┃
┡━━━━━━━━━━━━━╇━━━━━━━┩
│ 1962-01-01  │  421  │
│ 1962-05-01  │  245  │
│ 1962-09-01  │  239  │
│ 1962-03-01  │  215  │
└─────────────┴───────┘

      Mean Digit Ratio per Issue      
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Start issue ┃   Mean digit ratio   ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ 1962-01-01  │ 0.018023873033621005 │
│ 1962-03-01  │ 0.006935823165360167 │
│ 1962-05-01  │ 0.02027823914724555  │
│ 1962-09-01  │ 0.006525854030499938 │
└─────────────┴──────────────────────┘

Checking if actual issue: 100%|██████████| 166/166 [00:00<00:00, 2422.33it/s]


We are missing the following issues with this approach ['1962-09-01']

            Lowest Ten Percent of Tokens per Page             
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Start issue ┃ Page number ┃ Tokens per page ┃ Actual issue ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 1962-01-01  │      1      │       13        │     True     │
│ 1962-03-01  │     117     │        1        │     True     │
│ 1962-05-01  │     175     │       11        │     True     │
└─────────────┴─────────────┴─────────────────┴──────────────┘